In [2]:
import pandas as pd
import numpy as np

## Question 1:

### Which athlete has won the most Olympic medals?

In [3]:
od = pd.read_csv('data/olympics.csv', skiprows=4) # olympic data (od)

ath_count = od.groupby('Athlete').agg({'Medal':'count'}).sort_values(by='Medal').iloc[-1]

print(ath_count)

Medal    18
Name: LATYNINA, Larisa, dtype: int64


## Question 2:

### Which city has held the modern Games three times?

In [4]:
o_cities = od[['Edition', 'City']].drop_duplicates().groupby('City').agg({'Edition':'count'}).sort_values(by='Edition').iloc[-1]

print(o_cities) 

Edition    2
Name: Los Angeles, dtype: int64


Four-way tie, London held games in 2012 so that is the answer.

## Question 3: 

### Fourteen countries competed in the first modern Games in 1896. Which country topped the medal table?

In [5]:
od[od['Edition'] == 1896].groupby('NOC').agg({'Medal':'count'}).sort_values(by='Medal').iloc[-1]

Medal    52
Name: GRE, dtype: int64

## Question 4:

### Complete the sequence of 100m winners: Linford Christie, Donovan Bailey, Maurice Green ...

In [6]:
hm_winners = od[(od['Event'] == '100m') & (od['Medal'] == 'Gold') & (od['Gender'] == 'Men')].set_index('Athlete')
for i in range(hm_winners.shape[0]):
    if hm_winners.iloc[i - 1].name == 'GREENE, Maurice':
        print(hm_winners.iloc[i].name)

GATLIN, Justin


## Question 5:

### Which of these sports has never featured at the Olympics?

### Options: Motorcycle racing, Croquet, Squash, Swimming obstacle race

In [7]:
print(len(od[od['Sport'] == 'Croquet']))
print(len(od[od['Event'].str.contains('obstacle')]))
print(len(od[od['Sport'] == 'Squash']))
print(len(od[od['Sport'] == 'Motorcycle racing']))

8
3
0
0


Both Motorcycle racing and squash are possible options.

## Question 6:

### In which event did Baron Pierre de Coubertin, the president of the IOC and founder of the modern Olympic movement, win a gold medal at the 1912 Games?

In [8]:
od[od['Athlete'].str.contains('DE COUBERTIN')]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal


Name not in dataset, after looking it up he won in the 'Literature' event for his poem 'Ode to Sport' in the 1912 games.

## Question 7:

### Seventy-five of the 206 national Olympic committees have never won a medal at the Olympics. Which of these countries has the largest population?

In [9]:
pop = pd.read_csv('data/population.csv', skiprows=4)
pop = pop[['Country Name', '2008']]
ioc = pd.read_csv('data/ioccodes.csv')
od_countries = od['NOC'].drop_duplicates()
od_np = od_countries.to_numpy()
code_np = ioc['Int Olympic Committee code'].to_numpy()
no_medals = np.setdiff1d(code_np, od_np)
full_country_names = np.array(
    [np.char.strip(i, '*') for i in ioc[ioc['Int Olympic Committee code'].isin(no_medals)]['Country'].to_numpy()]
).flatten()
pop_no_medals = pop[pop['Country Name'].isin(full_country_names)].sort_values(by='2008')
print(pop_no_medals.iloc[-1]) 

Country Name     Bangladesh
2008            144304164.0
Name: 20, dtype: object


## Question 8

### Rio 2016 will be the third summer Games held in the southern hemisphere. Which city hosted the first?

In [29]:
cities = pd.read_csv('data/worldcities.csv')
cities_sh = cities[cities['lat'] < 0]       # Southern hemisphere cities
cities_sh_name_np = cities_sh['city'].to_numpy()
o_cities_series = od[['City', 'Edition']].drop_duplicates()
o_cities_series = o_cities_series.set_index('Edition').apply(
    lambda x: x.str.split(' / ').explode()
).reset_index()                             # Split 'Melborne / Stockholm' in two
sh_o_cities = o_cities_series[o_cities_series['City'].isin(cities_sh_name_np)]

print(sh_o_cities)

    Edition       City
12     1956  Melbourne
24     2000     Sydney


## Question 9:

### Golf is back at the Olympics this year. When did it last feature?

In [34]:
golf_df = od[od['Sport'] == 'Golf']
golf_df['Edition'].drop_duplicates().iloc[-1]

1904

## Question 10:

### Women first competed in the modern Olympics in 1900 but only in 'ladylike sports'. Which three?

In [40]:
woman_df = od[(od['Edition'] == 1900) & (od['Gender'] == 'Women')]
print(woman_df[['Sport','Event']].drop_duplicates())

      Sport          Event
416    Golf     individual
639  Tennis  mixed doubles
647  Tennis        singles


## Question 11: 

### Which of these athletes has won an Olympic gold medal?

##### Options: Floyd Mayweather Jr., Paula Radcliffe, Lionel Messi, Steve Cram

In [44]:
options = np.array(['MAYWEATHER, Floyd', 'RADCLIFFE, Paula', 'MESSI, Lionel', 'CRAM, Steve'])

gold_receipients = od[od['Medal'] == 'Gold']
gold_receipients[gold_receipients['Athlete'].isin(options)]

,City,Edition,Sport,Discipline,Athlete,NOC,Gender,Event,Event_gender,Medal
28224,Beijing,2008,Football,Football,"MESSI, Lionel",ARG,Men,football,M,Gold


## Question 12:

### In which sport do men and women compete against each other on equal terms?

In [49]:
gender = od[od['Event_gender'] == 'X']
gender['Sport'].drop_duplicates() # Shooting and Equestrianism are remaining options.

348      Equestrian
549         Sailing
635          Tennis
1845        Skating
13718      Shooting
21770     Badminton
Name: Sport, dtype: object